# Time Series Modeling (Smoothing Methods) Assignment

In [0]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import plotly.express as px

### Import the walmart stock prices data set.

In [0]:
df = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Time%20Series%20Analysis/Time%20Series%20-%20Day%203/walmart_stock_prices.csv')

In [0]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1259 entries, 0 to 1258
Data columns (total 7 columns):
date      1259 non-null object
open      1259 non-null float64
high      1259 non-null float64
low       1259 non-null float64
close     1259 non-null float64
volume    1259 non-null int64
Name      1259 non-null object
dtypes: float64(4), int64(1), object(2)
memory usage: 69.0+ KB


In [0]:
df['date'] = pd.to_datetime(df['date'])

In [0]:
df.head()

,date,open,high,low,close,volume,Name
0,2013-02-08,71.20,71.64,71.070,71.48,5906823,WMT
1,2013-02-11,71.25,71.51,70.530,71.40,6202534,WMT
2,2013-02-12,71.49,71.66,71.100,71.40,4761910,WMT
3,2013-02-13,71.29,71.70,71.210,71.39,3969807,WMT
4,2013-02-14,71.10,71.23,70.755,70.82,6820952,WMT


### Plot a line chart with the observed values (the daily closing prices).

In [0]:
def ilinechart(df,x,y,groups=None, title=''):
    fig = px.line(df,x=x,y=y, color=groups, title=title, template='none').update(layout=dict(title=dict(x=0.5)))
    
    # for item in range(len(fig.data)):
    #     fig.data[item].update(name=fig.data[item]['name'].split('=')[1])
    
    fig.show();

In [0]:
ilinechart(df, x='date', y='close')

### Choose 4 moving average time windows and add columns to the data set containing the values for each of them.

In [0]:
df['10_days'] = df['close'].rolling(window=10).mean()
df['25_days'] = df['close'].rolling(window=25).mean()
df['50_days'] = df['close'].rolling(window=50).mean()
df['100_days'] = df['close'].rolling(window=100).mean()

In [0]:
df.tail()

,date,open,high,low,close,volume,Name,10_days,25_days,50_days,100_days
1254,2018-02-01,105.96,106.56,105.18,105.52,6386836,WMT,106.612,102.8596,100.1202,92.5041
1255,2018-02-02,105.15,106.47,104.15,104.48,9253068,WMT,106.601,103.0684,100.2602,92.7581
1256,2018-02-05,103.40,104.61,97.09,100.09,12561594,WMT,106.065,103.0960,100.3316,92.9629
1257,2018-02-06,97.64,101.08,97.25,100.90,16413797,WMT,105.565,103.1820,100.4214,93.1733
1258,2018-02-07,100.50,104.59,100.01,102.85,11165223,WMT,105.271,103.3524,100.5460,93.4050


### Plot a multi-line chart that compares the 4 different simple moving averages.

In [0]:
%matplotlib inline

In [0]:
rolling_averages.tail()

,date,Time Window,Closing Price
5031,2018-02-01,100_days,92.5041
5032,2018-02-02,100_days,92.7581
5033,2018-02-05,100_days,92.9629
5034,2018-02-06,100_days,93.1733
5035,2018-02-07,100_days,93.4050


In [0]:
rolling_averages = pd.melt(df,id_vars='date', value_vars=['10_days', '25_days', '50_days', '100_days'], var_name='Time Window', value_name='Closing Price')
ilinechart(rolling_averages, 'date', 'Closing Price', groups='Time Window', title='Rolling Means')

### Write a function that calculates a weighted moving average. Take the weighted moving average of the observed values and store it in a new column in the dataframe.

In [0]:
def wma(df, field, window):
  weights = np.arange(1, window + 1)
  ma = df[field].rolling(window)
  wma = ma.apply(lambda x: np.dot(x, weights)/weights.sum(), raw=True)
  return wma

In [0]:
df['25_day_wma'] = wma(df, 'close', 25)
df['100_day_wma'] = wma(df, 'close', 100)

In [0]:
df.tail()

,date,open,high,low,close,volume,Name,10_days,25_days,50_days,100_days,25_day_wma,100_day_wma
1254,2018-02-01,105.96,106.56,105.18,105.52,6386836,WMT,106.612,102.8596,100.1202,92.5041,104.596523,97.424937
1255,2018-02-02,105.15,106.47,104.15,104.48,9253068,WMT,106.601,103.0684,100.2602,92.7581,104.721169,97.662083
1256,2018-02-05,103.40,104.61,97.09,100.09,12561594,WMT,106.065,103.0960,100.3316,92.9629,104.492062,97.807269
1257,2018-02-06,97.64,101.08,97.25,100.90,16413797,WMT,105.565,103.1820,100.4214,93.1733,104.323138,97.964440
1258,2018-02-07,100.50,104.59,100.01,102.85,11165223,WMT,105.271,103.3524,100.5460,93.4050,104.297600,98.156057


### Plot the weighted moving average in a multi-line plot with its simple moving average of the same length and evaluate the differences.

In [0]:
comparing_100_day_averages = pd.melt(df,id_vars='date', value_vars=['100_days', '100_day_wma'], var_name='Time Window', value_name='Closing Price')
ilinechart(comparing_100_day_averages, 'date', 'Closing Price', groups='Time Window', title='100 Day Comparison')

### Apply a simple exponential smoothing model to the observed values and store the results in a new column.

In [0]:
model = sm.tsa.ExponentialSmoothing(df['close']).fit()
comparing_100_day_averages['100_day_SExp'] = model.predict(100)

In [0]:
df.tail()

,date,open,high,low,close,volume,Name,10_days,25_days,50_days,100_days,25_day_wma,100_day_wma,100_Day_SExp
1254,2018-02-01,105.96,106.56,105.18,105.52,6386836,WMT,106.612,102.8596,100.1202,92.5041,104.596523,97.424937,106.629250
1255,2018-02-02,105.15,106.47,104.15,104.48,9253068,WMT,106.601,103.0684,100.2602,92.7581,104.721169,97.662083,105.547623
1256,2018-02-05,103.40,104.61,97.09,100.09,12561594,WMT,106.065,103.0960,100.3316,92.9629,104.492062,97.807269,104.506587
1257,2018-02-06,97.64,101.08,97.25,100.90,16413797,WMT,105.565,103.1820,100.4214,93.1733,104.323138,97.964440,100.199984
1258,2018-02-07,100.50,104.59,100.01,102.85,11165223,WMT,105.271,103.3524,100.5460,93.4050,104.297600,98.156057,100.882568


### Add the simple exponential smoothing values to your multi-line plot containing the simple and weighted moving average values and evaluate how they differ.

In [0]:
comparing_100_day_averages = pd.melt(df,id_vars='date', value_vars=['100_days', '100_day_wma', '100_Day_SExp'], var_name='Time Window', value_name='Closing Price')
ilinechart(comparing_100_day_averages, 'date', 'Closing Price', groups='Time Window', title='100 Day Comparison')

### Perform double and triple exponential smoothing and store their respective values in new columns.

In [0]:
double_model = sm.tsa.ExponentialSmoothing(df['close'], trend='add').fit()
comparing_100_day_averages['100_day_DExp'] = double_model.predict(100)

In [0]:
triple_model = sm.tsa.ExponentialSmoothing(df['close'], trend='add', seasonal='add', seasonal_periods=10).fit()
comparing_100_day_averages['100_day_TExp'] = triple_model.predict(100)

### Create a new multi-line plot showing the results of the three exponential smoothing methods.

In [0]:
comparing_100_day_averages['100_day_SExp'] = model.predict(100)
comparing_100_day_averages

,date,Time Window,Closing Price,100_day_DExp,100_day_TExp,100_day_SExp
0,2013-02-08,100_days,NaN,NaN,NaN,NaN
1,2013-02-11,100_days,NaN,NaN,NaN,NaN
2,2013-02-12,100_days,NaN,NaN,NaN,NaN
3,2013-02-13,100_days,NaN,NaN,NaN,NaN
4,2013-02-14,100_days,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
3772,2018-02-01,100_Day_SExp,106.629250,NaN,NaN,NaN
3773,2018-02-02,100_Day_SExp,105.547623,NaN,NaN,NaN
3774,2018-02-05,100_Day_SExp,104.506587,NaN,NaN,NaN
3775,2018-02-06,100_Day_SExp,100.199984,NaN,NaN,NaN


In [0]:
comparing_100_day_averages = pd.melt(comparing_100_day_averages,id_vars='date', value_vars=['100_day_SExp', '100_day_DExp', '100_day_TExp'], var_name='Time Window', value_name='Closing Price')
ilinechart(comparing_100_day_averages, 'date', 'Closing Price', groups='Time Window', title='Comparing Exponential Smoothing')

### Evaluate the performances of the three exponential smoothing methods by calculating their mean absolute error and their root mean squared error. Which one modeled the data best?

In [0]:
df['SExp_Diff'] = df['close'] - df['100_Day_SExp']
df['DExp_Diff'] = df['close'] - df['100_day_DExp']
df['TExp_Diff'] = df['close'] - df['100_day_TExp']

In [0]:
df['100_day_DExp'] = double_model.predict(100)
df['100_day_TExp'] = triple_model.predict(100)

In [0]:
df.tail()

,date,open,high,low,close,volume,Name,10_days,25_days,50_days,100_days,25_day_wma,100_day_wma,100_Day_SExp,100_day_DExp,100_day_TExp,SExp_Diff,DExp_Diff,TExp_Diff
1254,2018-02-01,105.96,106.56,105.18,105.52,6386836,WMT,106.612,102.8596,100.1202,92.5041,104.596523,97.424937,106.629250,106.656005,106.573602,-1.109250,-1.136005,-1.053602
1255,2018-02-02,105.15,106.47,104.15,104.48,9253068,WMT,106.601,103.0684,100.2602,92.7581,104.721169,97.662083,105.547623,105.574300,105.608491,-1.067623,-1.094300,-1.128491
1256,2018-02-05,103.40,104.61,97.09,100.09,12561594,WMT,106.065,103.0960,100.3316,92.9629,104.492062,97.807269,104.506587,104.533219,104.485199,-4.416587,-4.443219,-4.395199
1257,2018-02-06,97.64,101.08,97.25,100.90,16413797,WMT,105.565,103.1820,100.4214,93.1733,104.323138,97.964440,100.199984,100.229955,100.180763,0.700016,0.670045,0.719237
1258,2018-02-07,100.50,104.59,100.01,102.85,11165223,WMT,105.271,103.3524,100.5460,93.4050,104.297600,98.156057,100.882568,100.907524,100.941186,1.967432,1.942476,1.908814


In [0]:
print('Simple MAE:', df['SExp_Diff'].abs().mean(),
      'RMSE:', np.sqrt(np.mean(df['SExp_Diff']**2)))

print('Double MAE:', df['DExp_Diff'].abs().mean(),
      'RMSE:', np.sqrt(np.mean(df['DExp_Diff']**2)))

print('Triple MAE:', df['TExp_Diff'].abs().mean(),
      'RMSE:', np.sqrt(np.mean(df['TExp_Diff']**2)))

Simple MAE: 0.5490454029917842 RMSE: 0.836465698696353
Double MAE: 0.5482148875796643 RMSE: 0.8360934357714518
Triple MAE: 0.5473534994368037 RMSE: 0.8337189574081788
